In [1]:
%config Completer.use_jedi = False
from nilearn import datasets
from nilearn import image
from nilearn import plotting
import numpy as np

/gpfs/ysm/project/joormann/oad4/conda_envs/neuroAnalysis/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
datadir = '/gpfs/gibbs/pi/levy_ifat/Or/ROI/'
cort = datasets.atlas.fetch_atlas_harvard_oxford('cort-maxprob-thr50-2mm', data_dir=datadir)
subcort = datasets.atlas.fetch_atlas_harvard_oxford('sub-maxprob-thr50-2mm', data_dir=datadir)

In [22]:
# organize the images and the labels
images = [cort['maps'], subcort['maps']]
labelsBoth = np.hstack((cort['labels'][1:], subcort['labels'][1:]))
np.savetxt('labels_harvard_oxfordCombined.txt', labelsBoth, fmt='%s')

In [20]:
labelsBoth.shape

(69,)

In [10]:
#len(cort['labels'][1:])
len(subcort['labels'][1:])

21

In [4]:
len(cort['labels'])
len(subcort['labels'])
48+21

69

In [13]:
subcort['labels']

['Background',
 'Left Cerebral White Matter',
 'Left Cerebral Cortex ',
 'Left Lateral Ventrical',
 'Left Thalamus',
 'Left Caudate',
 'Left Putamen',
 'Left Pallidum',
 'Brain-Stem',
 'Left Hippocampus',
 'Left Amygdala',
 'Left Accumbens',
 'Right Cerebral White Matter',
 'Right Cerebral Cortex ',
 'Right Lateral Ventricle',
 'Right Thalamus',
 'Right Caudate',
 'Right Putamen',
 'Right Pallidum',
 'Right Hippocampus',
 'Right Amygdala',
 'Right Accumbens']

In [12]:
labelsBoth = np.array(labelsBoth[0], labelsBoth[1])
labelsBoth

TypeError: Field elements must be 2- or 3-tuples, got ''Background''

In [9]:
# only if we want to concatenate - not neccessary
atlas = image.load_img(images)
atlas = image.math_img('np.sum(img, axis=-1)', img=atlas)

In [11]:
atlas.to_filename('harvard_oxford_both.nii.gz')

In [ ]:
plotting.plot_roi(atlas)

## Extract time series


In [5]:
from nltools.data import Brain_Data
import glob
import os
import pandas as pd

In [ ]:
pd.show_versions()


In [7]:
#mask = Brain_Data(atlas)
mask_cort = Brain_Data(cort['maps']) # just cortical
mask_subcort = Brain_Data(subcort['maps']) # just subcort

In [ ]:
mask.plot()

In [8]:
# extract timeseries for the Shen parcellation (268 regions)
data_dir = '/gpfs/gibbs/pi/levy_ifat/Or/RCF_clips/results/Harvard_OxfordAtlas/' # where to save timeseries
base_dir = '/gpfs/gibbs/pi/levy_ifat/Or/RCF_clips/derivatives/fmriprep' # where to take them from

scan = 'ses-3'
file_list = glob.glob(os.path.join(base_dir, '*',f'{scan}', 'func', f'*denoise*hdf5'))
for f in file_list:
    print(f)
    sub = os.path.basename(f).split('_')[0]
    print(sub)
    data = Brain_Data(f)
    roi_cort = data.extract_roi(mask_cort)
    roi_sub = data.extract_roi(mask_subcort)
    roi = np.vstack((roi_cort, roi_sub)) # stacking both of the together
    pd.DataFrame(roi.T).to_csv(os.path.join(os.path.dirname(f), f"{sub}_{scan}_Average_ROI_ho_atlas.csv" ), index=False)
    np.save(os.path.join(data_dir, f"{sub}_{scan}_h_o_atlas" ), roi)

/gpfs/gibbs/pi/levy_ifat/Or/RCF_clips/derivatives/fmriprep/sub-1237/ses-3/func/sub-1237_ses-3_denoise_smooth6mm_task-movie_space-MNI152NLin2009cAsym_desc-preproc_bold.hdf5
sub-1237
/gpfs/gibbs/pi/levy_ifat/Or/RCF_clips/derivatives/fmriprep/sub-1099/ses-3/func/sub-1099_ses-3_denoise_smooth6mm_task-movie_space-MNI152NLin2009cAsym_desc-preproc_bold.hdf5
sub-1099
/gpfs/gibbs/pi/levy_ifat/Or/RCF_clips/derivatives/fmriprep/sub-1272/ses-3/func/sub-1272_ses-3_denoise_smooth6mm_task-movie_space-MNI152NLin2009cAsym_desc-preproc_bold.hdf5
sub-1272
/gpfs/gibbs/pi/levy_ifat/Or/RCF_clips/derivatives/fmriprep/sub-1350/ses-3/func/sub-1350_ses-3_denoise_smooth6mm_task-movie_space-MNI152NLin2009cAsym_desc-preproc_bold.hdf5
sub-1350
/gpfs/gibbs/pi/levy_ifat/Or/RCF_clips/derivatives/fmriprep/sub-1271/ses-3/func/sub-1271_ses-3_denoise_smooth6mm_task-movie_space-MNI152NLin2009cAsym_desc-preproc_bold.hdf5
sub-1271
/gpfs/gibbs/pi/levy_ifat/Or/RCF_clips/derivatives/fmriprep/sub-1269/ses-3/func/sub-1269_ses-3_d

In [ ]:
roi = np.vstack((roi_cort, roi_sub))
roi.shape